In [1]:
import os
import random
import shutil
from ultralytics import YOLO
import torch
import pandas as pd

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\pocsc\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
!unzip recipe_ingredients.zip -d recipe_ingredients

Streaming output truncated to the last 5000 lines.
  inflating: recipe_ingredients/train/labels/frame3510_jpg.rf.451d194ff3db97acaea6bc9df4a9d8e5.txt  
  inflating: recipe_ingredients/train/labels/frame3515_jpg.rf.3feeb5d7ce57d8b0ba60005280ac259c.txt  
  inflating: recipe_ingredients/train/labels/frame3515_jpg.rf.9b3b9d9cd1dd5052fc6cf920aa743305.txt  
  inflating: recipe_ingredients/train/labels/frame3520_jpg.rf.68e28230014e29cdd0d563b66c45f5b2.txt  
  inflating: recipe_ingredients/train/labels/frame3520_jpg.rf.6c9084c21d94e0ec31e7d5a2959ecbb1.txt  
  inflating: recipe_ingredients/train/labels/frame3525_jpg.rf.48f0abecebc41a5f49f3e53b6739f6f0.txt  
  inflating: recipe_ingredients/train/labels/frame3525_jpg.rf.7e96d77f826e9ccfc98c0aa21db0fce0.txt  
  inflating: recipe_ingredients/train/labels/frame3530_jpg.rf.0cb619d6eed3755790e187143b3cc561.txt  
  inflating: recipe_ingredients/train/labels/frame3530_jpg.rf.665d449fe54c51c27d04b4242bedd435.txt  
  inflating: recipe_ingredients/train/la

In [6]:
random.seed(42)

dataset_path = 'recipe_ingredients/'
data_yaml_path = os.path.join(dataset_path, 'data.yaml')

In [ ]:
def create_data_subset(source_dir, target_dir, percentage=0.25):
    """Create a subset of the data for faster training"""
    os.makedirs(target_dir, exist_ok=True)

    os.makedirs(os.path.join(target_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(target_dir, 'labels'), exist_ok=True)

    # All images in this dataset are only of type .jpg
    image_files = [f for f in os.listdir(os.path.join(source_dir, 'images'))
                   if f.endswith(('.jpg'))]

    subset_size = int(len(image_files) * percentage)

    selected_files = random.sample(image_files, subset_size)

    print(f"Creating subset with {subset_size} images from {len(image_files)} total images")

    for file in selected_files:
        src_img = os.path.join(source_dir, 'images', file)
        dst_img = os.path.join(target_dir, 'images', file)
        shutil.copy(src_img, dst_img)

        label_file = os.path.splitext(file)[0] + '.txt'
        src_label = os.path.join(source_dir, 'labels', label_file)
        dst_label = os.path.join(target_dir, 'labels', label_file)

        if os.path.exists(src_label):
            shutil.copy(src_label, dst_label)

print("Creating data subsets...")
create_data_subset(os.path.join(dataset_path, 'train'),
                  os.path.join(dataset_path, 'train_subset'), 0.25)
create_data_subset(os.path.join(dataset_path, 'valid'),
                  os.path.join(dataset_path, 'valid_subset'), 0.30)
create_data_subset(os.path.join(dataset_path, 'test'),
                  os.path.join(dataset_path, 'test_subset'), 0.20)

Creating data subsets...
Creating subset with 3957 images from 15829 total images
Creating subset with 244 images from 816 total images
Creating subset with 221 images from 1109 total images


In [ ]:
cwd = os.getcwd()
# Fetching "current working directory" cause Ultralytics
# had an issue w/ its default config looking for another path

In [ ]:
with open(data_yaml_path, 'r') as f:
    yaml_content = f.read()

yaml_content = yaml_content.replace('train: train/images', f'train: {cwd}/recipe_ingredients/train_subset/images')
yaml_content = yaml_content.replace('val: valid/images', f'val: {cwd}/recipe_ingredients/valid_subset/images')
yaml_content = yaml_content.replace('test: test/images', f'test: {cwd}/recipe_ingredients/test_subset/images')

subset_yaml_path = os.path.join(dataset_path, 'data_subset.yaml')
with open(subset_yaml_path, 'w') as f:
    f.write(yaml_content)

print(f"Created subset data.yaml at {subset_yaml_path}")

# This subset had a typo with tomato so we had to manually fix that typo
# and retrain the model

Created subset data.yaml at recipe_ingredients/data_subset.yaml


In [ ]:
model = YOLO('yolov8s.pt')
# Using a pretrained YOLOv8 model which works well when it comes to
# Object Detection

results = model.train(
    data=subset_yaml_path,
    epochs=10,
    imgsz=640,        
    batch=32,              
    patience=5,           
    device=0 if torch.cuda.is_available() else 'cpu',
    project='runs/train',
    name='ingredient_detector_subset',
    save=True,     
    plots=True                  
)

metrics = model.val()
print(f"Validation metrics: {metrics}")

# Test on a rand img from test subset
test_images = os.listdir(os.path.join(dataset_path, 'test_subset', 'images'))
if test_images:
    random_test_image = os.path.join(dataset_path, 'test_subset', 'images', random.choice(test_images))
    print(f"Testing on random image: {random_test_image}")
    results = model(random_test_image)

    results[0].save(filename=os.path.join('runs/train/ingredient_detector_subset', 'test_prediction.jpg'))
    print("Saved prediction visualization")

torch.save(model.model.state_dict(), 'YOLO_v8.pth')
# Renamed model in post production

Ultralytics 8.3.115 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=recipe_ingredients/data_subset.yaml, epochs=10, time=None, patience=5, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=runs/train, name=ingredient_detector_subset3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tr

train: Scanning /content/recipe_ingredients/train_subset/labels.cache... 3957 images, 1 backgrounds, 2 corrupt: 100%|██████████| 3957/3957 [00:00<?, ?it/s]

train: /content/recipe_ingredients/train_subset/images/3212_jpg.rf.0089184fb2f249b66fbbcfd6e0579876.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0556]
train: /content/recipe_ingredients/train_subset/images/3212_jpg.rf.82a641510ee5f57174f5d46016d89be5.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0556]
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 698.7±471.8 MB/s, size: 37.9 KB)


val: Scanning /content/recipe_ingredients/valid_subset/labels.cache... 244 images, 0 backgrounds, 0 corrupt: 100%|██████████| 244/244 [00:00<?, ?it/s]


...


10 epochs completed in 0.228 hours.
Optimizer stripped from runs/train/ingredient_detector_subset3/weights/last.pt, 22.5MB
Optimizer stripped from runs/train/ingredient_detector_subset3/weights/best.pt, 22.5MB

Validating runs/train/ingredient_detector_subset3/weights/best.pt...
Ultralytics 8.3.115 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,137,968 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:09<00:00,  2.32s/it]


                   all        244       2875      0.888      0.877      0.916      0.643
           bell_pepper         34         37      0.868      0.892      0.903      0.621
                carrot         27         30      0.927      0.842      0.924      0.605
               chicken         33        114      0.933      0.974      0.979      0.756
              cucumber         13         13       0.96          1      0.995       0.84
              eggplant          2          2      0.812          1      0.995      0.796
                garlic         95        157      0.925      0.943       0.95      0.666
                ginger        106        193      0.949      0.995      0.971      0.828
    green_chili_pepper         24         24      0.946      0.958      0.941      0.588
           green_onion          3          3      0.701      0.333      0.376      0.226
               kumquat         49        858      0.988      0.998      0.993      0.723
                 lemo

val: Scanning /content/recipe_ingredients/valid_subset/labels.cache... 244 images, 0 backgrounds, 0 corrupt: 100%|██████████| 244/244 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:08<00:00,  1.12s/it]


                   all        244       2875      0.889      0.874      0.916      0.645
           bell_pepper         34         37       0.87      0.892      0.903      0.623
                carrot         27         30      0.926      0.839      0.924      0.605
               chicken         33        114      0.934      0.974      0.979      0.754
              cucumber         13         13      0.961          1      0.995       0.84
              eggplant          2          2      0.816          1      0.995      0.796
                garlic         95        157      0.926      0.943       0.95      0.664
                ginger        106        193      0.949      0.995      0.971      0.829
    green_chili_pepper         24         24      0.948      0.958      0.941      0.587
           green_onion          3          3      0.706      0.333      0.376      0.226
               kumquat         49        858      0.988      0.998      0.993      0.724
                 lemo

We chose the YOLOv8 model because this particular dataset:

https://universe.roboflow.com/food-w4zm1/recipe-ingredients-cn

for the purposes of our app--identifying individual ingredients from an image of a group of ingredients--had exactly what we needed. An image of groups of ingredients, labeled with what ingredient they were and at what coordinates (boxed boundaries) they were located in the images.

We chose the particular YOLOv8 model, because, it seemed to be the latest version that supported this particular dataset for our particular task (object detection).

We spent a lot of time attempting to train our models on our own systems--and we managed to get it to recognize and "use" our GPUs, but we seemed to have run into bottleneck issues of sorts that failed to diagnose. Eventually, we used Google Colab, which, thankfully, does give us some limited usage with their Tesla 4 GPU, which trained our models quite fast.

We'd only trained/validated/tested on a subset of the dataset:

3957/15829 total images for training
244/816 total images for validation
221/1109 total images for testing

Because of the limited GPU usage provided by Google Colab.

We used image sizes of 640x640, because we didn't want them too small--as the model may have failed to properly recognize the images, and we didn't want them too large--as training may have taken too long.

Ultralytic's .val() metrics returned 4 values:

Box(P) -- Precision of bounding boxes; this measures the "accuracy" of the predicted boxes for the ingredients within an image = (0.878 final or 87.8%) 

R -- Recall; measures what % of objects are detected (0.879 final or 87.9%)

mAP50 -- Mean Average Precision (at IoU threshold 0.5); quantifies the overall performance of our model (0.907 final or 90.7%)

mAP50-95 -- Mean Average Precision avged over multiple IoU thresholds; more stringent/severe quantification of our model's performance (0.645 final or 64.5%)

Class Performance:

* High-performing classes:
    * Cucumber: 0.995 mAP50 or 99.5%
    * Tomato: 0.994 mAP50 or 99.4%
    * Kumquat: 0.993 mAP50 or 99.3%
    * Potato: 0.987 mAP50 or 98.7%
* Low-performing classes:
    * Green Onion: 0.376 mAP50 (only 3 instances in validation)

We could train some more to address issues like green onion, but again, Google Colab only provides us a few uses and its "reset schedule" is unpredictable as it is undocumented.